<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/hf_finetunning_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.3 MB/s 


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


 * page : https://huggingface.co/course/fr/chapter3/2?fw=tf
 * video :
   * https://www.youtube.com/watch?v=W_gMJF0xomE&ab_channel=HuggingFace
   * https://www.youtube.com/watch?v=P-rZWqcB6CE&ab_channel=HuggingFace
   * https://www.youtube.com/watch?v=7q5NyFT8REg&ab_channel=HuggingFace

## Intro

In [4]:

import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Même chose que précédemment
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    # J'ai attendu un cours de HuggingFace toute ma vie.
    "This course is amazing!",  # Ce cours est incroyable !
]
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))
batch_v2 = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
# on dirait que y'a pas réellement besoin d'empaquetter le résultat avec dict, trop con lol

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print(batch)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [6]:
print(batch_v2)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


In [8]:
print(model)
# Le model possède un type spécial transformers mais TF

In [9]:
# Ceci est nouveau
# il semble reprendre la terminologie de tensorflow avec .compile() et .train_on_batch()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
labels = tf.convert_to_tensor([1, 1])
model.train_on_batch(batch, labels)

2.324578417756129e-06

## HuggingFace Datasets



Le hub HuggingFace ne possède pas seulement des modèles pretrained mais également des datasets. Voir ici : https://huggingface.co/datasets . Pour télécharger en cache un jeu de données rien de plus simple, utilisez `load_dataset("glue", "mrpc")` du submodule `datasets`. L'output est alors un DatasetDict qui contient 3 entrées, une pour chaque set : train, validation, test. Chaque entrée contient un dataset dont les colonnes sont accessibles par leur nom

In [15]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
# Le dataset mrpc est un dataset inclus dans le benchmark appelée GLUE très utilisé dans la NLP
# Il s'agit de paires de phrases qui se suivent

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

#### Structure d'un DatasetDict

In [18]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [19]:
raw_datasets["train"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [20]:
raw_datasets["train"][1221]

{'idx': 1362,
 'label': 1,
 'sentence1': 'They were released to authorities after officers delivered the soda to the gunman , using a long stick to pass the six-pack through a door .',
 'sentence2': 'They were released to authorities after officers delivered a six-pack of Dr Pepper to the gunman , using a long stick to pass the soda through a door .'}

In [26]:
# Afin d'accéder aux codes de labelling
raw_datasets["train"].features
# not equivalent à 0
# equivalent à 1

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

## Preprocessing of Pairs

Le tokenizer peut prendre simplement un tuple comme paire de sentences

In [27]:
# Cas simple avec une paire

inputs = tokenizer(
    "This is the first sentence.", "This is the second one."
)  # "C'est la première phrase.", "C'est la deuxième."
inputs
# token_type_ids permet de distinguer les deux sentences

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

A noter qu'en plus de la méthode `.decode()`, on a la méthode `.convert_ids_to_tokens()`. **Rappel**, mieux vaut toujours utiliser le même checkpoint pour le modèle et pour le tokenizer

In [28]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Il est largement recommandé de travailler avec des données encapsulés dans des HuggingFace datasets car les données sont alors gérer sous Apache Arrow et de ce fait, n'est mis en RAM que les données requises. On va donc introduire la méthode `.map()` qui permet d'appliquer une fonction sur chaque observation d'un dataset. `batched=True` signifie au mapper d'appliquer la fonction par batch, ce qui rend l'opération plus efficace. Cependant la taille du batch on en sait rien pour le moment. Le mapper a pour effet d'ajouter aux features du datasetdict les nouvelles entrées produites par la fonction intégrée au mapper. Ceux-ci correspondent aux entrées du résultat du tokenizer à savoir `attention`, `input_ids` `input_token_type`

In [29]:
# La fonction qui sera appelé dans .map()
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Parameter 'function'=<function tokenize_function at 0x7fa940b8b320> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

## Dynamic Padding

On a volontairement reporté à plus tard le padding pour les raisons suivantes :
 * si on padd par rapport à la sentence la plus longue, on risque d'avoir un nombre de padded tokens très important, ce qui a pour effect un nombre d'opération inutile. Le moyen de bypasser ca, est de padder par rapport à la phrase la plus longue dans un batch donné
 * a noter qu'en cas d'usage de TPU, il vaut mieux ne pas faire ce dynamic padding, i.e. un padd définie par batch mais un seul padd pour l'ensemble des données d'apprentissage
 * La fonction responsable de l'aggrégation des observations dans un seul batch, i.e. un tensor représentant plusieurs observations, s'appelle une fonction collator. Il en existe des dizaines, nous allons utiliser une fonction particulièrement adapté qui s'appelle `DataCollatorWithPadding`


In [30]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
# Un batch de données sous forme de dict de 8 paires de phrases
# - label
# - input_ids
# - token_type_ids
# - attention mask

batch = data_collator(samples)
# un batch sous forme de dict de tensor au format tensorflow
# - chaque entrée a sur la première dimension la valeur 8

In [37]:
print(batch["input_ids"].shape)

(8, 67)


Evidemment la façon présentée au dessus n'est pas très optimale dans la pratique on fait la chose suivante : 
 * On convertit les transformers datasets en tensorflow dataset via la méthode .`to_tf_dataset()`
 * On se retrouve alors avec des tensorflow datasets qui seront utilisables lors de l'entraînement avec l'api tensorflow

In [38]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [39]:
print(tf_train_dataset)

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
